In [1]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine,text
from scipy import stats
import plotly.express as px
import toml
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import h5py
from functions import process_network_summary

%matplotlib inline
from IPython.display import display, HTML

pd.options.display.float_format = '{:0,.0f}'.format

In [2]:
df = process_network_summary()

# Truck VMT by County


In [3]:

df['medium_truck_vmt'] = df['@mveh']*df['length']
df['heavy_truck_vmt'] = df['@hveh']*df['length']
_df = df[['medium_truck_vmt','heavy_truck_vmt','county']].groupby('county').sum()[['medium_truck_vmt','heavy_truck_vmt']]
_df.loc['Total',:] = _df.sum()
_df = _df.reset_index()
_df.rename(columns={'county':'County', 'medium_truck_vmt': 'Medium Trucks', 'heavy_truck_vmt': 'Heavy Trucks'}, inplace=True)
_df


,County,Medium Trucks,Heavy Trucks
0,King,"1,956,010","1,417,579"
1,Kitsap,"145,247","90,383"
2,Outside Region,21,"22,280"
3,Pierce,"549,814","539,803"
4,Snohomish,"551,120","534,324"
5,Total,"3,202,212","2,604,370"


# Congestion by County

In [4]:
df['Medium and Heavy Trucks'] = df['heavy_truck_vmt']+df['medium_truck_vmt']

_df = df[df['tod'].isin(['5to6','6to7','7to8','8to9'])]
_df = _df.pivot_table(index='congestion_category',columns='county',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.loc['Total',:] = _df.sum()
_df

,King,Kitsap,Outside Region,Pierce,Snohomish
Light,"615,294","56,276","5,619","233,180","235,204"
Moderate,"132,697",719,NaN,"31,541","24,894"
Heavy,"93,776","1,216",NaN,"8,292","11,577"
Severe,"5,691",46,0,463,450
Total,"847,458","58,257","5,619","273,477","272,124"


In [5]:
_df = df[df['tod'].isin(['15to16','16to17','17to18'])]
_df = _df.pivot_table(index='congestion_category',columns='county',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.loc['Total',:] = _df.sum()
_df

,King,Kitsap,Outside Region,Pierce,Snohomish
Light,"353,001","32,776","3,301","135,647","134,459"
Moderate,"96,612",646,1,"19,814","19,783"
Heavy,"44,836",643,NaN,"4,420","5,108"
Severe,"2,881",44,0,335,304
Total,"497,330","34,110","3,303","160,216","159,654"


# Truck Miles on FGTS
Freight and Goods Transportation System
- T-1: More than 10 million tons per year
- T-2: 2 4 million to 10 million tons per year

In [6]:
pd.options.display.float_format = '{:0,.0f}'.format
df['Non-Truck'] = df['VMT']-df['Medium and Heavy Trucks']
_df = df[['medium_truck_vmt','heavy_truck_vmt','@fgts','Non-Truck']].melt(
    id_vars='@fgts', var_name='Vehicle Type')
_df = _df.pivot_table(index='Vehicle Type', columns='@fgts', 
                values='value', aggfunc='sum')
# _df = _df.reset_index()
_df = _df.rename_axis(None, axis=1)
_df.rename(columns={0:'Other Routes', 1: 'T-1', 2: 'T-2'}, inplace=True)
_df.rename(index={'heavy_truck_vmt': 'Heavy Trucks',
                 'medium_truck_vmt': 'Medium Trucks'}, inplace=True)
_df[['T-1','T-2','Other Routes']]

,T-1,T-2,Other Routes
Vehicle Type,,,
Non-Truck,"31,064,569","11,373,160","34,009,222"
Heavy Trucks,"2,086,270","171,871","346,228"
Medium Trucks,"1,778,537","486,469","937,206"


In [7]:
pd.options.display.float_format = '{:0,.1%}'.format
_df[['T-1','T-2','Other Routes']]/_df[['T-1','T-2','Other Routes']].sum()

,T-1,T-2,Other Routes
Vehicle Type,,,
Non-Truck,88.9%,94.5%,96.4%
Heavy Trucks,6.0%,1.4%,1.0%
Medium Trucks,5.1%,4.0%,2.7%


# Congestion on FGTS

In [8]:
pd.options.display.float_format = '{:0,.0f}'.format

# Congested Miles on FGTS versus other Routes
df['Medium and Heavy Trucks'] = df['@mveh']+df['@hveh']

_df = df[df['tod'].isin(['5to6','6to7','7to8','8to9'])]
_df = _df.pivot_table(index='congestion_category',columns='@fgts',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.rename(columns={0:'Other Routes', 1: 'T-1', 2: 'T-2'}, inplace=True)
_df_ = _df.copy()
_df.loc['Total',:] = _df.sum()
_df = _df[['T-1','T-2','Other Routes']]
_df

,T-1,T-2,Other Routes
Light,"2,191,663","832,153","1,767,582"
Moderate,"433,547","106,823","212,123"
Heavy,"277,850","93,918","192,351"
Severe,"30,383","34,826","80,070"
Total,"2,933,443","1,067,720","2,252,126"


In [9]:
pd.options.display.float_format = '{:0,.1%}'.format
_df_/_df_.sum()

,Other Routes,T-1,T-2
Light,78.5%,74.7%,77.9%
Moderate,9.4%,14.8%,10.0%
Heavy,8.5%,9.5%,8.8%
Severe,3.6%,1.0%,3.3%


In [10]:
pd.options.display.float_format = '{:0,.0f}'.format
# Congested Miles on FGTS versus other Routes
df['Medium and Heavy Trucks'] = df['@hveh']+df['@mveh']

_df = df[df['tod'].isin(['15to16','16to17','17to18'])]
_df = _df.pivot_table(index='congestion_category',columns='@fgts',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.rename(columns={0:'Other Routes', 1: 'T-1', 2: 'T-2'}, inplace=True)
_df_ = _df.copy()
_df.loc['Total',:] = _df.sum()
_df = _df[['T-1','T-2','Other Routes']]
_df

,T-1,T-2,Other Routes
Light,"1,242,747","484,820","996,027"
Moderate,"306,872","61,389","130,301"
Heavy,"142,514","52,119","117,759"
Severe,"18,176","23,878","48,788"
Total,"1,710,308","622,206","1,292,875"


In [11]:
pd.options.display.float_format = '{:0,.1%}'.format
_df_/_df_.sum()

,Other Routes,T-1,T-2
Light,77.0%,72.7%,77.9%
Moderate,10.1%,17.9%,9.9%
Heavy,9.1%,8.3%,8.4%
Severe,3.8%,1.1%,3.8%


# Equity

## Households Within 500' of Heavy Truck Volumes
Total number of households within 500' of T-1 and T-2 routes
- T-1: More than 10 million tons per year
- T-2: 2 4 million to 10 million tons per year

In [12]:
import polars as pl

# Intersect buffer with land use file
df_lu = pl.read_csv(r'..\..\..\..\outputs\landuse\parcels_urbansim.txt', separator=' ').to_pandas()

# Load as a geodataframe
gdf_lu = gpd.GeoDataFrame(
    df_lu, geometry=gpd.points_from_xy(df_lu.xcoord_p, df_lu.ycoord_p))
# Set the coordinate reference system (CRS) to match the land use data
gdf_lu.crs = 'EPSG:2285'

# Buffer the parcels at 500ft
gdf_lu['geometry'] = gdf_lu.buffer(500)

In [13]:

# Intersect this geography  with the network shapefile
gdf_network = gpd.read_file(r'..\..\..\..\inputs\scenario\networks\shapefiles\AM\AM_edges.shp')
# Do not include connectors since these are abstracted ul3==5; also remove weave links ul3==0 
gdf_network = gdf_network[~gdf_network.ul3.isin([0,5])]
# Truck network links are those that are in FGTS 1 or 2 system
gdf_network = gdf_network[gdf_network['FGTS'].isin([1,2])]

gdf_intersect = gpd.overlay(gdf_network, gdf_lu, how="intersection", keep_geom_type=False)

# Will need to relaculate the lengths since some were split across the regional geographies
gdf_intersect['new_length'] = gdf_intersect.geometry.length/5280.0

# filter out the polygon results and only keep lines
gdf_intersect = gdf_intersect[gdf_intersect.geometry.type.isin(['MultiLineString','LineString'])]

In [14]:
truck_parcels = gdf_intersect.groupby('parcelid').first()[['hh_p']].reset_index()


# Result should be the network components with some flags for the parcelid
# We can take the parcel information, join with parcel info and group
# from input_configuration import base_year
import toml
config = toml.load(os.path.join(os.getcwd(),r'../../../../configuration/input_configuration.toml'))

parcel_geog = pd.read_sql_table('parcel_'+config['base_year']+'_geography', 'sqlite:///../../../../inputs/db/'+config['db_name'])
df = truck_parcels.merge(parcel_geog,left_on='parcelid', right_on='ParcelID')

In [15]:
# Get the total number of households that in equtiy geograhpies
# Comprae the percent of those that are in the buffer versus those that are not
# For the 4 equity groups, perform the calc and add as a table
results_df = pd.DataFrame()
for col, name in {'racial_geog_vs_reg_total': 'Race',
                  'poverty_geog_vs_reg_total': 'Poverty',
                  'elderly_geog_vs_reg_total': 'Elderly',
                  'youth_geog_vs_reg_total': 'Youth'}.items():
    _df = df[[col,'hh_p']].groupby(col).sum()[['hh_p']]
    _df['equity_group'] = name
    results_df = pd.concat([results_df,_df])
results_df = results_df.reset_index()
results_df = results_df[results_df['index']>=0]

## Total Households Within 500' of T-1/T-2 Routes by Equity Group

In [16]:
_df = results_df.pivot_table(index='index', columns='equity_group', values='hh_p', aggfunc='sum')
_df.index = ['Households Not in Equity Geography', 'Households in Equity Geography']
pd.options.display.float_format = '{:0,.0f}'.format
_df.astype('float')

equity_group,Elderly,Poverty,Race,Youth
Households Not in Equity Geography,"118,343","91,169","75,091","124,528"
Households in Equity Geography,"68,553","95,727","111,805","62,368"


In [17]:
pd.options.display.float_format = '{:0,.1%}'.format
_df/_df.sum()

equity_group,Elderly,Poverty,Race,Youth
Households Not in Equity Geography,63.3%,48.8%,40.2%,66.6%
Households in Equity Geography,36.7%,51.2%,59.8%,33.4%


In [63]:
pd.options.display.float_format = '{:0,.0f}'.format

# results_df.groupby(['equity_group','index']).sum()[['HH_P']]
_df = results_df[["equity_group","hh_p","index"]].pivot_table(index='equity_group',columns='index',values='hh_p')
_df.rename(columns={0:'Not In Equity Group', 1: 'In Equity Group'}, inplace=True)
_df.index.name = None
_df.columns.name = None
_df['Total Households'] = _df.sum(axis=1)
_df_buffer = _df.copy()

df_lu_tot = df_lu[['parcelid','hh_p']].merge(parcel_geog,left_on='parcelid', right_on='ParcelID')
# df_lu_tot['region'] = 1
results_df_tot = pd.DataFrame()

for col, name in {'racial_geog_vs_reg_total': 'Race',
                  'poverty_geog_vs_reg_total': 'Poverty',
                  'elderly_geog_vs_reg_total': 'Elderly',
                  'youth_geog_vs_reg_total': 'Youth',
#                   'region': 'Regional Total',
                 }.items():
    _df = df_lu_tot[['hh_p',col]].groupby(col).sum()[['hh_p']]
    _df['equity_group'] = name
    results_df_tot = pd.concat([results_df_tot, _df])
results_df_tot = results_df_tot.reset_index()
pd.options.display.float_format = '{:0,.0f}'.format

results_df_tot = results_df_tot[results_df_tot['index'] >= 0]
_df = results_df_tot.pivot_table(index='equity_group',columns='index',values='hh_p')
_df.rename(columns={0:'Not in Equity Group', 1: 'In Equity Group'}, inplace=True)
_df.index.name = None
_df.columns.name = None
_df['Total Households'] = _df.sum(axis=1)
_df_tot = _df.copy()
_df = _df_tot.merge(_df_buffer, left_index=True, right_index=True, suffixes=['_tot','_buffer'])

pd.options.display.float_format = '{:0,.1%}'.format
_df_compare = pd.DataFrame(_df['In Equity Group_buffer']/_df['In Equity Group_tot'], columns=['% of Total Households in Buffer'])
_df_compare.loc['All Households'] = (_df['Total Households_buffer']/_df['Total Households_tot'])['Elderly']
_df_compare

,% of Total Households in Buffer
Elderly,8.3%
Poverty,13.8%
Race,14.1%
Youth,8.0%
All Households,10.8%
